In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from datacleaner import autoclean

In [47]:
# Read in data
data = pd.read_csv('train.csv')
#data = autoclean(data)
print data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)None


In [48]:
# delete columns that aren't obviously useful
data = data.drop(['PassengerId', 'Ticket', 'Cabin'], axis=1)

In [49]:
# Assign catagorical columns integer values
data.ix[data.Sex == 'male', 'Sex'] = 0
data.ix[data.Sex == 'female', 'Sex'] = 1
data.Sex = data.Sex.astype('int64')

In [50]:
#print data.Name[0:100]
data.ix[[('Mr.' not in x) and ('Mrs.' not in x) and ('Master.' not in x) and ('Miss.' not in x) for x in data.Name], 'Name'] = '-1'
data.ix[[('Mr.' in x) for x in data.Name], 'Name'] = '0'
data.ix[[('Mrs.' in x) for x in data.Name], 'Name'] = '1'
data.ix[[('Master.' in x) for x in data.Name], 'Name'] = '2'
data.ix[[('Miss.' in x) for x in data.Name], 'Name'] = '3'
data.Name = data.Name.astype('int64')

In [51]:
data["Embarked"] = data["Embarked"].fillna("S")
data.ix[data.Embarked == 'C', 'Embarked'] = 0
data.ix[data.Embarked == 'Q', 'Embarked'] = 1
data.ix[data.Embarked == 'S', 'Embarked'] = 2
data.Embarked = data.Embarked.astype('int64')

In [52]:
# Fill missing Ages with median of name categories
data.Age[data.Name==-1] = data.Age[data.Name==-1].fillna(data.Age[data.Name==-1].median())
data.Age[data.Name==0] = data.Age[data.Name==0].fillna(data.Age[data.Name==0].median())
data.Age[data.Name==1] = data.Age[data.Name==1].fillna(data.Age[data.Name==1].median())
data.Age[data.Name==2] = data.Age[data.Name==2].fillna(data.Age[data.Name==2].median())
data.Age[data.Name==3] = data.Age[data.Name==3].fillna(data.Age[data.Name==3].median())
data.Age = data.Age.astype('int64')

In [53]:
data.Fare = data.Fare.astype('int64')
print data.info()
data.hist('Age')
plt.show()
# Clean NaN
#data = data.fillna(data.mean(), inplace=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 9 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Name        891 non-null int64
Sex         891 non-null int64
Age         891 non-null int64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null int64
Embarked    891 non-null int64
dtypes: int64(9)None


In [60]:
# Regularize
#print data.ix[:,'Pclass':]
data.ix[:,'Pclass':] = (data.ix[:,'Pclass':] - data.ix[:,'Pclass':].mean()) / data.ix[:,'Pclass':].std()

In [61]:
X = data.ix[:,'Pclass':].as_matrix()
Y = data.Survived.values
kf = KFold(n_splits=2)
for train_index, test_index in kf.split(X):
  #X_train = X[train_index]
  X_train, X_test = X[train_index], X[test_index]
  Y_train, Y_test = Y[train_index], Y[test_index]

In [74]:
# Decision tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, Y_train)
#Y_pred = clf.predict(data.Surv)
clf.score(X_test, Y_test)

0.75730337078651688

In [83]:
# Random forest
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
#Y_pred = random_forest.predict(X_test)
random_forest.score(X_test, Y_test)

0.78426966292134837

In [84]:
# Boosted tree
boost = AdaBoostClassifier()
boost.fit(X_train, Y_train)
boost.score(X_test, Y_test)

0.80449438202247192